## Classificação de Documentos com doc2Vec


Doc2vec é um algoritmo não supervisionado para gerar vetores para frases, parágrafos ou documentos (Representações distribuídas de sentenças e documento). Trata-se de um conceito que foi apresentado em 2014 por Le & Mikilov, veja neste [artigo](https://arxiv.org/abs/1405.4053). Este algoritmo é uma adaptação do word2vec, sendo que os vetores gerados pelo doc2vec podem ser usados para tarefas como encontrar semelhanças entre sentenças , parágrafos ou documentos.

Um vetor de documento é uma representação abstrata de comprimento variável do significado contextual de um determinado tipo de documento. Assim como um vetor de palavras, é o produto do processo de treinamento para uma rede neural, onde a entrada é tipicamente um termo one-hot encoded codificado a partir do vocabulário do modelo e a saída é uma distribuição de probabilidade para palavras na próxima janela de contexto. 


Neste mini-projeto utilizaremos o conjunto de dados da BBC [dataset](http://mlg.ucd.ie/datasets/bbc.html). Este conjunto de dados é composto de 2225 documentos do site de notícias da BBC correspondendo a histórias em cinco áreas distintas: (negócios, entretenimento, política, esporte, tecnologia).


## Dataset

Para essa tarefa, usaremos um conjunto de arquivos de texto já organizado. Este conjunto de dados é composto de 2225 documentos que correspondem a artigos de notícias da BBC, representando 5 tópicos distintos:

    * Negócios
    * Entretenimento
    * Política
    * Esportes
    * Tecnologia
    
Um exemplo de documento da categoria Tecnologia:

'UK net users leading TV downloads British TV viewers lead the trend of illegally downloading US shows from the net, according to research. New episodes of 24, Desperate Housewives and Six Feet Under, appear on the web hours after they are shown in the US, said a report. Web tracking company Envisional said 18% of downloaders were from within the UK and that downloads of TV programmers had increased by 150% in the last year....'

Usaremos 250 documentos de cada categoria. Nosso vocabulário será de tamanho 25.000. Além disso, cada documento será representado por uma tag - para fins de visualização. Por exemplo, o 50º documento da seção Entretenimento será representado como entretenimento-50. Deve-se notar que este é um conjunto de dados muito pequeno comparado ao grande corpora de texto que está sendo analisado em aplicações do mundo real. No entanto, este pequeno exemplo é adequado no momento para ver o poder das Word embeddings.


Usaremos a seguinte estratégia:

1. Extrair os dados de todos os arquivos de texto e aprender as word embeddings.
2. Extrair conjuntos randômicos de documentos dos word embeddings já treinados.
3. Preparar os dados para os modelos de treinamento e avaliação doc2Vec
4. Contruir o vocabulário através de um modelo doc2vec - Distributed Bag of Words (DBOW)
5. Aplicar um modelo de Regressão Logística para classificação dos documentos

In [6]:
import csv
from sklearn.metrics import classification_report
from itertools import islice, chain
import os
import numpy as np
import nltk
from nltk.corpus import stopwords
import collections
from IPython.display import Image
import random
import zipfile
from nltk.stem import SnowballStemmer
from string import punctuation
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from tqdm import tqdm
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
%matplotlib inline

In [7]:
url = 'http://mlg.ucd.ie/files/datasets/'

def maybe_download(filename, expected_bytes):
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Encontrado e verificado %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Não foi possível verificar o arquivo ' + filename + '. Você pode fazer o donwload via browser?')
    return filename

filename = maybe_download('bbc-fulltext.zip', 2874078)

Encontrado e verificado bbc-fulltext.zip


## Lendo os dados com pré-processamento utilizando a biblioteca NLTK

Efetua a leitura dos dados, depois converte o texto para letras minúscula e o converte em tokens usando a biblioteca nltk. Temos duas funções `read_data`, que lê `files_to_read_for_topic` de cada tópico e `read_test_data`, que separa 20 documentos de cada tópicos para fins de teste. 

In [23]:
def read_data(filename):
    """
      Extrai artigos até um determinado limite em um arquivo zip como uma lista de palavras
      e pré-processa usando a biblioteca nltk python
      """
    data = [[],[]]
    train_data = {}
    files_to_read_for_topic = 250
    topics = ['business','entertainment','politics','sport','tech']
    with zipfile.ZipFile(filename) as z:
        parent_dir = z.namelist()[0]
        for t in topics:
            print('\tConcluída a leitura de dados para o tópico: ', t)
            for fi in range(1,files_to_read_for_topic):
                with z.open(parent_dir + t + '/'+ format(fi,'03d')+'.txt') as f:
                    file_string = f.read().decode('latin-1')
                    file_string = file_string.lower()
                    file_string = nltk.word_tokenize(file_string)
                    # Atribui a classe aos arquivos
                    data[0].append(file_string)
                    data[1].append(t)
                    """ Atribui o tópico ao documento """
                    train_data[t+'-'+str(fi)] = file_string
               
    return data, train_data

def read_test_data(filename):
    """
    Extrai artigos até um determinado limite em um arquivo zip como uma lista de palavras
    e pré-processa usando a biblioteca nltk python
    """
    test_data = {}
    files_to_read_for_topic = 350
    topics = ['business','entertainment','politics','sport','tech']
    with zipfile.ZipFile(filename) as z:
        parent_dir = z.namelist()[0]
        for t in topics:
            print('\tConcluída a leitura de dados para o tópico: ',t)            
            for fi in np.random.randint(1,files_to_read_for_topic,(20)).tolist():
                with z.open(parent_dir + t + '/'+ format(fi,'03d')+'.txt') as f:
                    file_string = f.read().decode('latin-1')
                    file_string = file_string.lower()
                    file_string = nltk.word_tokenize(file_string) 
                    """ Atribui o tópico ao documento """
                    test_data[t+'-'+str(fi)] = file_string
                
    return test_data

print('Processando dados de treinamento...\n')
words, train_words = read_data(filename)
"""
  Como os documentos estão organizados por tópicos, decidi aplicar um "shuffle", para forçar que o dicionário seja montado 
  contendo os tópicos de forma aleatória, assegurando que não haja nenhum tipo de "vício" na construção do modelo.
"""
list_train_words = list(map(tuple, train_words.items()))
random.shuffle(list_train_words)
train_words = dict(list_train_words)

print('\nProcessando dados de teste...\n')

test_words = read_test_data(filename)

list_test_words = list(map(tuple, test_words.items()))
random.shuffle(list_test_words)
test_words = dict(list_test_words)


Processando dados de treinamento...

	Concluída a leitura de dados para o tópico:  business
	Concluída a leitura de dados para o tópico:  entertainment
	Concluída a leitura de dados para o tópico:  politics
	Concluída a leitura de dados para o tópico:  sport
	Concluída a leitura de dados para o tópico:  tech

Processando dados de teste...

	Concluída a leitura de dados para o tópico:  business
	Concluída a leitura de dados para o tópico:  entertainment
	Concluída a leitura de dados para o tópico:  politics
	Concluída a leitura de dados para o tópico:  sport
	Concluída a leitura de dados para o tópico:  tech


## Construindo os Dicionários
Para entender cada um desses elementos, vamos também assumir o texto "Eu gosto de ir à escola"

* `dictionary`: mapeia uma palavra para um ID (i.e. {Eu:0, gosto:1, de:2, ir:3, à:4, escola:5})
* `reverse_dictionary`: mapeia um ID para uma palavra (i.e. {0:Eu, 1:gosto, 2:de, 3:ir, 4:à, 5:escola}
* `count`: Lista de elementos (palavra, frequência) (i.e. [(Eu,1),(gosto,1),(de,2),(ir,1),(à,1),(escola,1)]
* `data` : Contém a string de texto que lemos, onde palavras são substituídas por IDs de palavras (i.e. [0, 1, 2, 3, 2, 4])

Também introduzimos um token especial adicional chamado `UNK` para indicar que palavras raras são muito raras para serem usadas.

In [25]:
vocabulary_size = 25000
Words = []
def build_dataset(words):
    for word in words[0]:
        Words.extend(word)    
    count = [['UNK', -1]]
    count.extend(collections.Counter(Words).most_common(vocabulary_size - 1))

    # Dicionário
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    
    data = list()
    unk_count = 0
    
    for word in Words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']            
            unk_count = unk_count + 1
            
        data.append(word)

    count[0][1] = unk_count
    
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    assert len(dictionary) == vocabulary_size

    return data, count, dictionary, reverse_dictionary

def build_dataset_with_existing_dictionary(words, dictionary):
    '''
    Aqui usamos essa função para converter strings de palavras em IDs com um determinado dicionário
    '''
    data = list()
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
        data.append(word)
    return data

# Processando dados de treino
data, count, dictionary, reverse_dictionary = build_dataset(words)

train_data = {}

for k,v in train_words.items():
    print('Construindo o dataset de teste para o documento ', k)
    train_data[k] = build_dataset_with_existing_dictionary(train_words[k],dictionary)

# Processando dados de teste

test_data = {}

for k,v in test_words.items():
    print('Construindo o dataset de teste para o documento ', k)
    test_data[k] = build_dataset_with_existing_dictionary(test_words[k],dictionary)
    
print('\nPalavras mais comuns (+UNK)', count[:5])
print('\nAmostra de dados', data[:10])
print('\nChaves: ', test_data.keys())
print('\nItems: ', test_data.items())

# Removemos para liberar memória no computador. Não precisamos mais desses objetos.
del words  
#del test_words

Construindo o dataset de teste para o documento  sport-211
Construindo o dataset de teste para o documento  entertainment-67
Construindo o dataset de teste para o documento  business-60
Construindo o dataset de teste para o documento  politics-215
Construindo o dataset de teste para o documento  entertainment-110
Construindo o dataset de teste para o documento  politics-81
Construindo o dataset de teste para o documento  tech-207
Construindo o dataset de teste para o documento  tech-47
Construindo o dataset de teste para o documento  sport-178
Construindo o dataset de teste para o documento  sport-14
Construindo o dataset de teste para o documento  politics-41
Construindo o dataset de teste para o documento  business-13
Construindo o dataset de teste para o documento  sport-228
Construindo o dataset de teste para o documento  sport-139
Construindo o dataset de teste para o documento  politics-232
Construindo o dataset de teste para o documento  entertainment-60
Construindo o dataset de

Construindo o dataset de teste para o documento  politics-66
Construindo o dataset de teste para o documento  politics-195
Construindo o dataset de teste para o documento  sport-137
Construindo o dataset de teste para o documento  politics-110
Construindo o dataset de teste para o documento  politics-22
Construindo o dataset de teste para o documento  entertainment-180
Construindo o dataset de teste para o documento  politics-210
Construindo o dataset de teste para o documento  sport-217
Construindo o dataset de teste para o documento  sport-56
Construindo o dataset de teste para o documento  tech-123
Construindo o dataset de teste para o documento  politics-168
Construindo o dataset de teste para o documento  business-54
Construindo o dataset de teste para o documento  politics-13
Construindo o dataset de teste para o documento  tech-71
Construindo o dataset de teste para o documento  politics-75
Construindo o dataset de teste para o documento  tech-141
Construindo o dataset de teste 

Construindo o dataset de teste para o documento  sport-155
Construindo o dataset de teste para o documento  politics-245
Construindo o dataset de teste para o documento  politics-119
Construindo o dataset de teste para o documento  sport-90
Construindo o dataset de teste para o documento  tech-115
Construindo o dataset de teste para o documento  business-79
Construindo o dataset de teste para o documento  politics-10
Construindo o dataset de teste para o documento  politics-189
Construindo o dataset de teste para o documento  business-229
Construindo o dataset de teste para o documento  sport-43
Construindo o dataset de teste para o documento  sport-175
Construindo o dataset de teste para o documento  entertainment-158
Construindo o dataset de teste para o documento  sport-132
Construindo o dataset de teste para o documento  sport-89
Construindo o dataset de teste para o documento  entertainment-111
Construindo o dataset de teste para o documento  entertainment-61
Construindo o dataset

Construindo o dataset de teste para o documento  politics-9
Construindo o dataset de teste para o documento  business-140
Construindo o dataset de teste para o documento  business-90
Construindo o dataset de teste para o documento  politics-46
Construindo o dataset de teste para o documento  tech-5
Construindo o dataset de teste para o documento  sport-166
Construindo o dataset de teste para o documento  politics-102
Construindo o dataset de teste para o documento  sport-85
Construindo o dataset de teste para o documento  entertainment-241
Construindo o dataset de teste para o documento  entertainment-79
Construindo o dataset de teste para o documento  business-94
Construindo o dataset de teste para o documento  politics-30
Construindo o dataset de teste para o documento  business-56
Construindo o dataset de teste para o documento  business-43
Construindo o dataset de teste para o documento  tech-244
Construindo o dataset de teste para o documento  business-99
Construindo o dataset de 

Construindo o dataset de teste para o documento  business-185
Construindo o dataset de teste para o documento  sport-125
Construindo o dataset de teste para o documento  tech-61
Construindo o dataset de teste para o documento  tech-182
Construindo o dataset de teste para o documento  politics-240
Construindo o dataset de teste para o documento  business-133
Construindo o dataset de teste para o documento  sport-243
Construindo o dataset de teste para o documento  politics-38
Construindo o dataset de teste para o documento  tech-134
Construindo o dataset de teste para o documento  politics-62
Construindo o dataset de teste para o documento  tech-81
Construindo o dataset de teste para o documento  politics-198
Construindo o dataset de teste para o documento  tech-235
Construindo o dataset de teste para o documento  sport-21
Construindo o dataset de teste para o documento  tech-153
Construindo o dataset de teste para o documento  entertainment-153
Construindo o dataset de teste para o doc

...

Construindo o dataset de teste para o documento  sport-229
Construindo o dataset de teste para o documento  politics-202
Construindo o dataset de teste para o documento  sport-68
Construindo o dataset de teste para o documento  politics-153
Construindo o dataset de teste para o documento  entertainment-7
Construindo o dataset de teste para o documento  tech-35
Construindo o dataset de teste para o documento  entertainment-141
Construindo o dataset de teste para o documento  entertainment-160
Construindo o dataset de teste para o documento  tech-221
Construindo o dataset de teste para o documento  entertainment-77
Construindo o dataset de teste para o documento  tech-204
Construindo o dataset de teste para o documento  politics-69
Construindo o dataset de teste para o documento  entertainment-10
Construindo o dataset de teste para o documento  sport-233
Construindo o dataset de teste para o documento  business-132
Construindo o dataset de teste para o documento  sport-104
Construindo o 

Construindo o dataset de teste para o documento  politics-105
Construindo o dataset de teste para o documento  tech-68
Construindo o dataset de teste para o documento  entertainment-200
Construindo o dataset de teste para o documento  entertainment-145
Construindo o dataset de teste para o documento  sport-206
Construindo o dataset de teste para o documento  entertainment-80
Construindo o dataset de teste para o documento  tech-13
Construindo o dataset de teste para o documento  business-186
Construindo o dataset de teste para o documento  politics-141
Construindo o dataset de teste para o documento  politics-244
Construindo o dataset de teste para o documento  sport-117
Construindo o dataset de teste para o documento  sport-28
Construindo o dataset de teste para o documento  entertainment-244
Construindo o dataset de teste para o documento  business-246
Construindo o dataset de teste para o documento  entertainment-134
Construindo o dataset de teste para o documento  tech-38
Construin

Construindo o dataset de teste para o documento  tech-231
Construindo o dataset de teste para o documento  politics-15
Construindo o dataset de teste para o documento  sport-112
Construindo o dataset de teste para o documento  entertainment-69
Construindo o dataset de teste para o documento  sport-152
Construindo o dataset de teste para o documento  sport-150
Construindo o dataset de teste para o documento  tech-199
Construindo o dataset de teste para o documento  tech-34
Construindo o dataset de teste para o documento  sport-220
Construindo o dataset de teste para o documento  politics-194
Construindo o dataset de teste para o documento  tech-31
Construindo o dataset de teste para o documento  business-88
Construindo o dataset de teste para o documento  politics-134
Construindo o dataset de teste para o documento  tech-49
Construindo o dataset de teste para o documento  business-204
Construindo o dataset de teste para o documento  entertainment-243
Construindo o dataset de teste para 

Construindo o dataset de teste para o documento  business-143
Construindo o dataset de teste para o documento  politics-45
Construindo o dataset de teste para o documento  politics-217
Construindo o dataset de teste para o documento  entertainment-103
Construindo o dataset de teste para o documento  tech-85
Construindo o dataset de teste para o documento  entertainment-270
Construindo o dataset de teste para o documento  tech-42
Construindo o dataset de teste para o documento  tech-255

Palavras mais comuns (+UNK) [['UNK', 907], ('the', 28312), ('.', 21345), (',', 18517), ('to', 13509)]

Amostra de dados ['ad', 'sales', 'boost', 'time', 'warner', 'profit', 'quarterly', 'profits', 'at', 'us']

Chaves:  dict_keys(['business-205', 'sport-309', 'tech-19', 'sport-171', 'sport-269', 'sport-264', 'entertainment-316', 'sport-150', 'entertainment-273', 'entertainment-232', 'tech-207', 'tech-336', 'tech-75', 'politics-160', 'entertainment-61', 'sport-242', 'politics-313', 'business-227', 'entert

Items:  dict_items([('business-205', ['orange', 'colour', 'clash', 'set', 'for', 'court', 'a', 'row', 'over', 'the', 'colour', 'orange', 'could', 'hit', 'the', 'courts', 'after', 'mobile', 'phone', 'giant', 'orange', 'launched', 'action', 'against', 'a', 'new', 'mobile', 'venture', 'from', 'easyjet', "'s", 'founder', '.', 'orange', 'said', 'it', 'was', 'starting', 'proceedings', 'against', 'the', 'easymobile', 'service', 'for', 'trademark', 'infringement', '.', 'easymobile', 'uses', 'easygroup', "'s", 'orange', 'branding', '.', 'founder', 'stelios', 'haji-ioannou', 'has', 'pledged', 'to', 'contest', 'the', 'action', '.', 'the', 'move', 'comes', 'after', 'the', 'two', 'sides', 'failed', 'to', 'come', 'to', 'an', 'agreement', 'after', 'six', 'months', 'of', 'talks', '.', 'orange', 'claims', 'the', 'new', 'low-cost', 'mobile', 'service', 'has', 'infringed', 'its', 'rights', 'regarding', 'the', 'use', 'of', 'the', 'colour', 'orange', 'and', 'could', 'confuse', 'customers', '-', 'known', 'a

## Preparação dos dados para criação do vocabulário no doc2vec

In [27]:
# Converte de Dicionário para lista
data_train = [ [k,v] for k, v in train_data.items() ]
data_train = np.array(data_train)
data_test = [ [k,v] for k, v in test_data.items() ]
data_test = np.array(data_test)

In [28]:
# função para Identificar as classes dos documentos e preparando os dados para o algoritimo
def prepara_dados(data):
    datax = [[],[]]
    for x in range(data.shape[0]):
        s = data[x][0]
        s = s.split("-")
        datax[0].append(s[0])
        datax[1].append(data[x][1])                
    return datax

data_train = prepara_dados(data_train)
data_test = prepara_dados(data_test)

In [29]:
def label_sentences(corpus, topics):
    """
    A implementação do Doc2Vec da Gensim exige que cada documento / parágrafo tenha um rótulo associado a ele.
    Fiz isso usando o método TaggedDocument, etiquetando com a própria classe do documento.
    """
   
    labeled = []
    tags = np.unique(topics, return_counts=False)
    for i, v in enumerate(corpus):
        label = [s for s in tags if topics[i] in s]
        doc =  " ".join(str(x) for x in v)
        labeled.append(TaggedDocument(doc.split(), label))
    return labeled
X_train = label_sentences(np.array(data_train[1]), data_train[0])
X_test  = label_sentences(np.array(data_test[1]), data_test[0])

In [86]:
len(X_train)

1245

In [87]:
len(X_test)

97

## Modelos de Treinamento e Avaliação doc2Vec

Primeiramente instanciamos um modelo doc2vec - Distributed Bag of Words (DBOW). Na arquitetura word2vec, nós temos os algoritmos  “continuous bag of words” (CBOW) e “skip-gram” (SG), já na arquitetura doc2vec, os algoritmos correspondentes são “distributed memory” (DM) e “distributed bag of words” (DBOW).

O DBOW é o modelo doc2vec análogo ao modelo Skip-gram do word2vec. Os vetores de parágrafos são obtidos pelo treinamento de uma rede neural na tarefa de prever uma distribuição de probabilidade de palavras em um parágrafo, dada uma palavra aleatoriamente amostrada do parágrafo.

In [195]:
Image(url = 'Doc2Vec.png')

In [31]:
# Instanciando um modelo Doc2Vec com um vetor de 128 palavras

model_dbow = Doc2Vec(dm=0, vector_size=128, window=10, negative=5, cbow_mean=1, min_count=1, alpha=0.1, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(X_train)])


# Alicando 50 iterações sobre o corpus de treinamento.

for epoch in range(50):
    model_dbow.train(utils.shuffle([x for x in tqdm(X_train)]), total_examples=len(X_train), epochs=10)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████████████████████████████████████████████████████████████████████████████████| 1245/1245 [00:00<?, ?it/s]


## Construindo os Vetores de Recurso para o Classificador

In [32]:
%%time

# Dados de treino

train_targets, train_regressors = zip(
    *[(doc.tags[0], model_dbow.infer_vector(doc.words, alpha=0.1, min_alpha=0.065, epochs=100)) for doc in X_train])

Wall time: 45.3 s


In [33]:
%%time

# Dados de teste

test_targets, test_regressors = zip(
    *[(doc.tags[0], model_dbow.infer_vector(doc.words, alpha=0.1, min_alpha=0.065, epochs=100)) for doc in X_test])

Wall time: 3.75 s


## Treinando o Classificador de Regressão Logística.

In [34]:
# Aplicando de Regressão Logistica
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_regressors, train_targets)
y_pred = logreg.predict(test_regressors)
print('accuracy %s' % accuracy_score(y_pred, test_targets))
print(classification_report(test_targets, y_pred,target_names=test_targets))

accuracy 0.9896907216494846


C:\Users\anton\Anaconda3\envs\PLN\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 5, does not match size of target_names, 97
  .format(len(labels), len(target_names))


               precision    recall  f1-score   support

     business       1.00      0.95      0.97        20
        sport       1.00      1.00      1.00        20
         tech       0.95      1.00      0.97        19
        sport       1.00      1.00      1.00        19
        sport       1.00      1.00      1.00        19

  avg / total       0.99      0.99      0.99        97



In [107]:
test_targets

('business',
 'politics',
 'entertainment',
 'sport',
 'business',
 'sport',
 'sport',
 'business',
 'politics',
 'business',
 'entertainment',
 'business',
 'business',
 'tech',
 'tech',
 'sport',
 'business',
 'politics',
 'tech',
 'politics',
 'tech',
 'tech',
 'entertainment',
 'tech',
 'tech',
 'business',
 'politics',
 'sport',
 'entertainment',
 'entertainment',
 'politics',
 'sport',
 'tech',
 'business',
 'politics',
 'tech',
 'politics',
 'politics',
 'tech',
 'politics',
 'tech',
 'sport',
 'business',
 'politics',
 'politics',
 'business',
 'tech',
 'politics',
 'business',
 'politics',
 'sport',
 'tech',
 'politics',
 'tech',
 'politics',
 'entertainment',
 'tech',
 'sport',
 'entertainment',
 'business',
 'entertainment',
 'tech',
 'sport',
 'sport',
 'tech',
 'politics',
 'entertainment',
 'sport',
 'business',
 'entertainment',
 'sport',
 'sport',
 'sport',
 'tech',
 'entertainment',
 'business',
 'entertainment',
 'politics',
 'entertainment',
 'tech',
 'entertainment'

In [100]:
len(test_targets)

97

In [35]:
acuracia = np.sum(y_pred == test_targets) / len(test_targets)

In [36]:
print(acuracia)

0.9896907216494846


In [106]:
y_pred

array(['business', 'politics', 'entertainment', 'sport', 'business',
       'sport', 'sport', 'business', 'politics', 'business',
       'entertainment', 'business', 'business', 'tech', 'tech', 'sport',
       'business', 'politics', 'tech', 'politics', 'tech', 'tech',
       'entertainment', 'tech', 'tech', 'business', 'politics', 'sport',
       'entertainment', 'entertainment', 'politics', 'sport', 'tech',
       'business', 'politics', 'tech', 'politics', 'politics', 'tech',
       'politics', 'tech', 'sport', 'business', 'politics', 'politics',
       'business', 'tech', 'politics', 'business', 'politics', 'sport',
       'tech', 'politics', 'tech', 'politics', 'entertainment', 'tech',
       'sport', 'entertainment', 'business', 'entertainment', 'tech',
       'sport', 'sport', 'tech', 'politics', 'entertainment', 'sport',
       'business', 'entertainment', 'sport', 'sport', 'sport', 'tech',
       'entertainment', 'business', 'entertainment', 'politics',
       'entertainment',

## Resultado

Fiquei impressionado com precisão apresentada nos dados de teste, onde este modelo atingiu uma acurácia de 0.9896907216494846, demostrando que doc2vec é uma excelente ferramenta para tarefas de classificação de documentos.

## Referencias

[Curso de Processamento de Linguagem Natural e Reconhecimento de Voz da Formação Inteligência Artificial - Datascienceacademy](https://www.datascienceacademy.com.br)

[BBC Datasets](http://mlg.ucd.ie/datasets/bbc.html)

[Gemsin](https://radimrehurek.com/gensim/models/keyedvectors.html)

[Distributed Representations of Words and Phrases and their Compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

[Distributed Representations of Sentences and Documents](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)

[A gentle introduction to Doc2Vec](https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e)

[Gensim Doc2Vec Tutorial on the IMDB Sentiment Dataset](https://github.com/RaRe-Technologies/gensim/blob/3c3506d51a2caf6b890de3b1b32a8b85f7566ca5/docs/notebooks/doc2vec-IMDB.ipynb)

[Document classification with word embeddings tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre/blob/master/ipynb_with_output/Document%20classification%20with%20word%20embeddings%20tutorial%20-%20with%20output.ipynb)